In [1]:
import pandas as pd
from e2e_system.reproder import _ground_truth
import numpy as np


In [3]:
df = pd.read_csv("data/acl/sections/labelled/zeroshot/header_plus_content.csv", index_col=0)


In [4]:
label_df = df.groupby("repo").agg(
    {"label": lambda x: list(set(list(x)))}).reset_index()
label_df["score"] = label_df["label"].apply(lambda x: 0 if len(x) <= 3 else 1)


In [5]:
label_df


repo  \
0     0xhesch/clat-cross-lingual-annotation-transfer   
1                      0xnurl/signaling-auto-encoder   
2                    11zhouxuan/multi_grained_kd_ner   
3                12kleingordon34/NLP_masters_project   
4                            131250208/infextraction   
...                                              ...   
6707                            zzcoolj/corpus2graph   
6708                           zzsfornlp/nnpgdparser   
6709                            zzsfornlp/znmt-merge   
6710                                     zzshou/RCAM   
6711                    zzshou/amr-data-augmentation   

                                                  label  score  
0                                        [introduction]      0  
1                               [requirements, results]      0  
2                                             [results]      0  
3                                [evaluation, training]      0  
4         [evaluation, training, requirements, results]      1  
...                                                 ...    ...  
6707                                     [requirements]      0  
6708      [evaluation, introduction, training, results]      1  
6709    [introduction, training, requirements, results]      1  
6710                   [pretrained_model, introduction]      0  
6711  [pretrained_model, introduction, evaluation, r...      1  

[6712 rows x 3 columns]

In [6]:
from tqdm.auto import tqdm
from util.file_helper import get_readme_with_repo_name
from util.readme_parser import ReadmeSectionParser

for idx, row in tqdm(label_df.iterrows()):
    readme = get_readme_with_repo_name("acl", row.repo)
    if readme is None:
        continue
    sections = ReadmeSectionParser("row.repo", readme).parse_sections()
    readme = ReadmeSectionParser.merge_sections(pd.DataFrame(sections))
    label_df.loc[idx, "readme"] = readme


0it [00:00, ?it/s]

In [7]:
label_df.dropna(inplace=True)
label_df.rename(columns={"readme": "text", "score": "label"}, inplace=True)
label_df.to_csv(
    "data/acl/sections/labelled/zeroshot/hierarchical_binary_train.csv", index=False)
